# Import module

In [2]:
import os
from pyvi import ViTokenizer
from tqdm import tqdm
import re

# Tiền xử lý dữ liệu

In [3]:
ROOT_PATH = 'Train_Full'
tree_path = os.walk(ROOT_PATH)
file_list = []

for (dirpath, dirname, filenames) in tree_path:
    if dirpath == ROOT_PATH: 
        continue
    for filename in filenames:
        file_list.append(os.path.join(dirpath, filename))

In [4]:
file_list[:3]

['Train_Full\\Chinh tri Xa hoi\\XH_NLD_T_ (8997).txt',
 'Train_Full\\Chinh tri Xa hoi\\XH_NLD_T_ (8999).txt',
 'Train_Full\\Chinh tri Xa hoi\\XH_NLD_T_ (9001).txt']

In [5]:
# Hàm để loại bỏ các ký tự biểu cảm
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

# Chuyển thành chữ viết thường, bỏ biểu bảm và các ký tự đặc biệt
# sau đó sử dụng Vitokenizer là bộ tokennize cho tiếng Việt có hỗ trợ word segmentation
def make_clean(string, to_tokens=True):
    res = string.lower()
    res = remove_emoji(res)
    res = re.sub(r'[!“”"#$%&\()*+,-./:;<=>?@[\]^_`{|}~]', " ", res)
    res = re.sub(r'\s+', ' ', res).strip()
    res = ViTokenizer.tokenize(res)
    if to_tokens:
        res = res.split()
    return res

make_clean('Tôi     là(*(*%$sinh (*&viên', to_tokens=True)

['tôi', 'là', 'sinh_viên']

In [6]:
LOOKBACK_LENGTH = 50 # sử dụng 50 từ trước để dự đoán 1 từ tiếp theo
sequences = []
for filepath in tqdm(file_list[:1000]): # phần demo này mình chỉ sử dụng 1,000 file đầu tiên mà mình đọc được để chạy nhẹ máy (do sức mạnh máy không đáp ứng hết cả bộ dữ liệu - mọi người có thể sử dụng colab và bật GPU để chạy nhanh hơn nhé!)
    with open(filepath, encoding='utf-16') as f:
        p = f.read()
        tokens = make_clean(p, to_tokens=True)
        for i in range(len(tokens) - LOOKBACK_LENGTH):
            seq = tokens[i:i+LOOKBACK_LENGTH+1]
            sequences.append(' '.join(seq))

100%|██████████| 1000/1000 [00:07<00:00, 132.66it/s]


In [9]:
# lưu dữ liệu vào file train.txt
with open('train.txt', 'w', encoding='utf-16') as f:
    for seq in sequences:
        f.write(seq+'\n')